# Wide and Deep Model for Movie Recommendation

### Prerequisite
* `tensorflow` (or setup gpu venv -- TODO: link to SETUP guide)

In this example, we utilize TensorFlow's higher level Estimator API to build [wide-and-deep model](https://ai.googleblog.com/2016/06/wide-deep-learning-better-together-with.html) for movie recommendation. For more details about how wide-and-deep learning works, see linked [paper](https://arxiv.org/abs/1606.07792).

In [3]:
import sys
sys.path.append("../../")

import itertools

import tensorflow as tf
import pandas as pd

from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import (
    rmse, mae, rsquared, exp_var,
    map_at_k, ndcg_at_k, precision_at_k, recall_at_k
)

In [4]:
from tensorflow.python.client import device_lib

devices = device_lib.list_local_devices()
[x.name for x in devices]

['/device:CPU:0', '/device:GPU:0']

### Data loading

In [5]:
MOVIELENS_DATA_SIZE = '100k'

In [6]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=['UserId','MovieId','Rating','Timestamp'],
    # TODO For now, not using genres YET
    load_genres=False
)
data.head()


,UserId,MovieId,Rating,Timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


### Feature embedding

Wide and deep model utilizes two different types of feature set: 1) a wide set of cross-producted features to capture how the co-occurrence of a query-item feature pair correlates with the target label or rating, and 2) a deep, lower-dimensional embedding vectors for every query and item.

In [7]:
""" Wide columns """

_HASH_BUCKET_SIZE = 1000

# Distinct users and items
user_list = data['UserId'].unique()
item_list = data['MovieId'].unique()

user_id = tf.feature_column.categorical_column_with_vocabulary_list(
    'UserId', user_list)
item_id = tf.feature_column.categorical_column_with_vocabulary_list(
    'MovieId', item_list)

# Wide columns and deep columns.
base_columns = [user_id, item_id]
crossed_columns = [
    tf.feature_column.crossed_column(
          ['UserId', 'MovieId'], hash_bucket_size=_HASH_BUCKET_SIZE),
    # TODO maybe add genre here too
#       tf.feature_column.crossed_column(
#           [age_buckets, 'education', 'occupation'],
#           hash_bucket_size=_HASH_BUCKET_SIZE),
]

# TODO try w/o base_columns
wide_columns = base_columns + crossed_columns

# TODO check features by:
# features = tf.parse_example(..., features=make_parse_example_spec(columns))
# dense_tensor = input_layer(features, columns)


In [5]:
""" Deep columns """

# Rule of thumb for embedding_dimensions =  number_of_categories ** 0.25
USER_EMBEDDING_DIM = int(len(user_list) ** 0.25) # or 16
ITEM_EMBEDDING_DIM = int(len(item_list) ** 0.25) # or 64
print("Embedding {} users to {}-dim vector".format(len(user_list), USER_EMBEDDING_DIM))
print("Embedding {} items to {}-dim vector".format(len(item_list), ITEM_EMBEDDING_DIM))

# Convert a categorical feature, e.g. UserId or MovieId, into a lower-dimensional embedding vectors
user_embedding = tf.feature_column.embedding_column(
    categorical_column=user_id,
    dimension=USER_EMBEDDING_DIM,
    max_norm=USER_EMBEDDING_DIM**.5)

item_embedding = tf.feature_column.embedding_column(
    categorical_column=item_id,
    dimension=ITEM_EMBEDDING_DIM,
    max_norm=ITEM_EMBEDDING_DIM**.5)

timestamp = tf.feature_column.numeric_column('Timestamp')

# TODO numeric_column (w/ shape)
# genres = tf.feature_column.numeric_column(
#     'Genre', shape=(NUM_GENRES,), dtype=tf.uint8)

deep_columns = [user_embedding, item_embedding, timestamp]  # TODO , genres]
wide_columns = []  # TODO cross product transformation of user and item

Embedding 943 users to 5-dim vector
Embedding 1682 items to 6-dim vector


Tran and test data split

In [8]:
train, test = python_random_split(data, ratio=0.75, seed=123)

train_x = train.copy()
train_y = train_x.pop('Rating')
test_x = test.copy()
test_y = test_x.pop('Rating')

print(train_x.head())
print("\nLabels:")
print(train_y.head())

       UserId  MovieId  Timestamp
31450     496      136  876066424
42809      64      101  889740225
52419     158      471  880132513
45663     198      652  884209569
50696     749      121  878847645

Labels:
31450    1.0
42809    2.0
52419    4.0
45663    3.0
50696    3.0
Name: Rating, dtype: float64


### Model preparation

Model selection
* `wide` - Linear model
* `deep` - DNN model
* `wide_deep` - Linear combination of the linear and DNN models

(TODO)Model type: `regressor` or `classifier`

In [9]:
# 'wide', 'deep', or 'wide_deep' 
MODEL_TYPE = 'wide'
# DNN hidden nodes
HIDDEN_UNITS = [256, 256, 256, 128]
# Model checkpoints folder
MODEL_DIR = './models'

In [10]:
# TODO set run config if needed
if MODEL_TYPE == 'wide':
    if len(wide_columns) == 0:
        raise ValueError("No features have defined for the 'wide' model")
    model = tf.estimator.LinearRegressor(  # LinearClassifier(
        model_dir=MODEL_DIR,
        feature_columns=wide_columns,
    )
elif MODEL_TYPE == 'deep':
    if len(deep_columns) == 0:
        raise ValueError("No features have defined for the 'deep' model")
    model = tf.estimator.DNNRegressor(  # DNNClassifier(
        model_dir=MODEL_DIR,
        feature_columns=deep_columns,
        hidden_units=HIDDEN_UNITS,
        optimizer=tf.train.AdamOptimizer(),
#         activation_fn=tf.nn.sigmoid,
#         dropout=0.3,
#         loss_reduction=tf.losses.Reduction.MEAN,
#         batch_norm=False
    )
elif MODEL_TYPE == 'wide_deep':
    if len(wide_columns) == 0 and len(deep_columns) == 0:
        raise ValueError("No features have defined for the 'wide_deep' model")
    model = tf.estimator.DNNLinearCombinedRegressor(  # DNNLinearCombinedClassifier(
        model_dir=MODEL_DIR,
        # wide settings
        linear_feature_columns=wide_columns,
        # deep settings
        dnn_feature_columns=deep_columns,
        dnn_hidden_units=HIDDEN_UNITS,
    )
else:
    raise ValueError("Model type should be either 'wide', 'deep', or 'wide_deep'")


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc9dddf7d68>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


### Training

In [11]:
# Maybe should set tf.estimator.RunConfig to run on GPU?

BATCH_SIZE = 256
NUM_EPOCHS = 50

train_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=train_x,
    y=train_y,
    batch_size=BATCH_SIZE,
    num_epochs=NUM_EPOCHS,
    shuffle=True,
    num_threads=1
)

model.train(input_fn=train_input_fn)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./models/model.ckpt.
INFO:tensorflow:loss = 3695.0, step = 1
INFO:tensorflow:global_step/sec: 142.574
INFO:tensorflow:loss = 251.51472, step = 101 (0.703 sec)
INFO:tensorflow:global_step/sec: 198.346
INFO:tensorflow:loss = 207.60385, step = 201 (0.504 sec)
INFO:tensorflow:global_step/sec: 206.948
INFO:tensorflow:loss = 219.08542, step = 301 (0.483 sec)
INFO:tensorflow:global_step/sec: 156.129
INFO:tensorflow:loss = 212.32654, step = 401 (0.645 sec)
INFO:tensorflow:global_step/sec: 120.729
INFO:tensorflow:loss = 184.71173, step = 501 (0.824 sec)
INFO:tensorflow:global_step/sec: 187.374
INFO:tensorflow:loss = 171.0518, step = 601 (0.535 sec)
INFO:tensorflow:global_step/sec: 203.944
INFO:tensorflow:loss = 216.3405, step = 701 (0.489 sec)
INFO:tensorflow:global_step/sec: 200.185
INFO:tensorflow:loss = 210.99043, step = 801 (0.499 sec)
INFO:tensorflow:global_step/sec: 194.016
INFO:tensorflow:loss = 205

INFO:tensorflow:global_step/sec: 201.746
INFO:tensorflow:loss = 206.86874, step = 8301 (0.494 sec)
INFO:tensorflow:global_step/sec: 206.264
INFO:tensorflow:loss = 183.67978, step = 8401 (0.486 sec)
INFO:tensorflow:global_step/sec: 198.629
INFO:tensorflow:loss = 231.63138, step = 8501 (0.503 sec)
INFO:tensorflow:global_step/sec: 204.207
INFO:tensorflow:loss = 198.64767, step = 8601 (0.488 sec)
INFO:tensorflow:global_step/sec: 195.908
INFO:tensorflow:loss = 200.86711, step = 8701 (0.511 sec)
INFO:tensorflow:global_step/sec: 183.698
INFO:tensorflow:loss = 222.76509, step = 8801 (0.544 sec)
INFO:tensorflow:global_step/sec: 118.037
INFO:tensorflow:loss = 185.83377, step = 8901 (0.847 sec)
INFO:tensorflow:global_step/sec: 151.528
INFO:tensorflow:loss = 207.75725, step = 9001 (0.660 sec)
INFO:tensorflow:global_step/sec: 201.986
INFO:tensorflow:loss = 211.99585, step = 9101 (0.496 sec)
INFO:tensorflow:global_step/sec: 206.051
INFO:tensorflow:loss = 183.69858, step = 9201 (0.484 sec)
INFO:tenso

### Testing

To evaluate our model, we first evaluate the baseline performance by predicting a user's rating as a simple average of his/her previous ratings. Then, we predict the ratings by using the wide-deep model we trained. Finally, we also generate top-k movie recommentation for each user and test the performance.

#### Baseline performance

In [43]:
baseline = train.groupby(['UserId'])['Rating'].mean()
baseline = baseline.to_frame().reset_index()
baseline.head()
# baseline = pd.merge(test, baseline.to_frame(['UserId', 'prediction']), on=['UserId'], how='inner')
# baseline.head()
# baseline.sort_values(by=['UserId']).head()

,UserId,Rating
0,1,3.655172
1,2,3.711111
2,3,2.756757
3,4,4.111111
4,5,2.939850


In [31]:
cols = {
    'col_user': 'UserId',
    'col_item': 'MovieId',
    'col_rating': 'Rating',
    'col_prediction': 'prediction',
}

eval_rmse = rmse(test, baseline, **cols)
eval_mae = mae(test, baseline, **cols)
eval_rsquared = rsquared(test, baseline, **cols)
eval_exp_var = exp_var(test, baseline, **cols)

print("RMSE:\t\t%f" % eval_rmse,
      "MAE:\t\t%f" % eval_mae,
      "rsquared:\t%f" % eval_rsquared,
      "exp var:\t%f" % eval_exp_var, sep='\n')

KeyError: 'rating'

#### NN model performance

In [13]:
test_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=test_x,
    y=test_y,
    num_epochs=1,
    shuffle=False
)

result = model.evaluate(input_fn=test_input_fn, steps=None)

print(result)

INFO:tensorflow:Starting evaluation at 2018-12-15-23:44:15
INFO:tensorflow:Restoring parameters from ./models/model.ckpt-14649
INFO:tensorflow:Finished evaluation at 2018-12-15-23:44:16
INFO:tensorflow:Saving dict for global step 14649: average_loss = 0.91905546, global_step = 14649, loss = 117.22646
{'average_loss': 0.91905546, 'loss': 117.22646, 'global_step': 14649}


1. Item rating prediction

In [22]:
pred_list = [p['predictions'][0] for p in list(model.predict(input_fn=test_input_fn))]
predictions = test.copy()
predictions['prediction']  = pd.Series(pred_list).values
print(predictions.head())

INFO:tensorflow:Restoring parameters from ./models/model.ckpt-14649
       UserId  MovieId  Rating  Timestamp  prediction
42083     600      651     4.0  888451492    4.182480
71825     607      494     5.0  883879556    3.595096
99535     875     1103     5.0  876465144    4.357851
47879     648      238     3.0  882213535    4.077431
36734     113      273     4.0  875935609    3.902378


In [15]:
eval_rmse = rmse(test, predictions, **cols)
eval_mae = mae(test, predictions, **cols)
eval_rsquared = rsquared(test, predictions, **cols)
eval_exp_var = exp_var(test, predictions, **cols)

print("RMSE:\t\t%f" % eval_rmse,
      "MAE:\t\t%f" % eval_mae,
      "rsquared:\t%f" % eval_rsquared,
      "exp var:\t%f" % eval_exp_var, sep='\n')

RMSE:		0.958674
MAE:		0.755316
rsquared:	0.285917
exp var:	0.285921


2. Recommend k items

1) Remove seen items and 2) add timestamp info

In [16]:
# Get the cross join of all user-item pairs and score them.
user_item_col = ['UserId', 'MovieId']
user_item_list = list(itertools.product(user_list, item_list))
users_items = pd.DataFrame(user_item_list, columns=user_item_col)
print("Before excude seen items:", len(users_items))

# Remove seen items (items in the train set)
users_items_exclude_train = users_items.loc[
    ~users_items.set_index(user_item_col).index.isin(train.set_index(user_item_col).index)
]
print("After excude seen items:", len(users_items_exclude_train))

# Add timestamp info
users_items_exclude_train = pd.merge(test, users_items_exclude_train,
                                     on=user_item_col, how='outer')
users_items_exclude_train.drop('Rating', axis=1, inplace=True)
users_items_exclude_train.fillna(test['Timestamp'].max(), inplace=True) 
print(users_items_exclude_train.head())

Before excude seen items: 1586126
After excude seen items: 1511126
   UserId  MovieId    Timestamp
0     600      651  888451492.0
1     607      494  883879556.0
2     875     1103  876465144.0
3     648      238  882213535.0
4     113      273  875935609.0


In [17]:
reco_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=users_items_exclude_train,
    num_epochs=1,
    shuffle=False
)

reco = list(model.predict(input_fn=reco_input_fn))
reco_list = [p['predictions'][0] for p in reco]
users_items_exclude_train['prediction']  = pd.Series(reco_list).values
users_items_exclude_train.head()


INFO:tensorflow:Restoring parameters from ./models/model.ckpt-14649


,UserId,MovieId,Timestamp,prediction
0,600,651,888451492.0,4.182480
1,607,494,883879556.0,3.595096
2,875,1103,876465144.0,4.357851
3,648,238,882213535.0,4.077431
4,113,273,875935609.0,3.902378


In [18]:
k = 10
eval_map = map_at_k(test, users_items_exclude_train, k=k, **cols)
eval_ndcg = ndcg_at_k(test, users_items_exclude_train, k=k, **cols)
eval_precision = precision_at_k(test, users_items_exclude_train, k=k, **cols)
eval_recall = recall_at_k(test, users_items_exclude_train, k=k, **cols)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.000414
NDCG:	0.005410
Precision@K:	0.006787
Recall@K:	0.002281
